In [ ]:
import re, subprocess, pyodbc 
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import connstr  
import pandas as pd

In [ ]:
engine = create_engine(connstr)
conn = engine.connect()
query = """ \
    SELECT FP.ASPX , FP.PK_FormPage,  FP.Text, MCK.* FROM vw_MetricsCompositeKeys MCK 
    INNER JOIN fsma_FormPages FP ON FP.PK_FormPage=MCK.FK_FormPage
    WHERE PK_FORM = '2021-A-EINSTEIN' AND PK_FormPage <> 2072
""" 
df = pd.read_sql(query,con=conn)  
df
alt = ''
for i,r in df.iterrows():
    text = re.sub('Section \d:' , f'Section {i+1}:' , r['Text'])
    alt = alt + f"""
    UPDATE fsma_FormPages SET Text = '{text}', sortpos = {i+1}, ASPX = 'EINSTEIN/2021/2021_A_EINSTEIN_{i}.aspx'
    WHERE PK_FormPage = {r['PK_FormPage']}
    """ 
conn.close()
print( f'{alt}' )


In [ ]:
engine = create_engine(connstr)
conn = engine.connect()
query = """ \
    SELECT QPK FROM vw_MetricsCompositeKeys MCK 
    INNER JOIN fsma_FormPages FP ON FP.PK_FormPage=MCK.FK_FormPage
    WHERE PK_FORM = '2021-A-EINSTEIN' AND PK_FormPage <> 2072
""" 
df = pd.read_sql(query,con=conn)  
df
alt = ''
for i,r in df.iterrows(): 
    alt = alt + f"""
    UPDATE fsma_Questions SET identifier_text = '{i+1}' WHERE PK_Question = {r['QPK']}""" 
conn.close()
print( f'{alt}' )

In [ ]:
secs = [
      {'s':1, 'name':'EinsteinPublicIP'}
    , {'s':2, 'name':'EinsteinUnannounced'}
    , {'s':3, 'name':'EinsteinBGP'}
    , {'s':4, 'name':'EinsteinUnmonitoredTraffic'}
    , {'s':5, 'name':'EinsteinTAPS'} 
    , {'s':7, 'name':'EinsteinNetChannelsDS'}
    , {'s':8, 'name':'EinsteinNAT'}
]

In [107]:

lst = [] 
for d in secs:
    df = pd.read_csv(f"C:\\Users\\Tim\\Documents\\EINSTEIN\\ss\\{d['s']}.csv")
    # D:\dev\CyberScope\CyberScopeBranch\CSwebdev\database\Archive\Old DBUpdates\DB_Update_7.27_2020_A_EINSTEIN_SAOP.sql
    df['SECTION'] = d['s']
    df['TABLE_NAME'] = d['name']
    df['COLUMN_NAME'] = df['Column Headings'].apply(lambda s: re.sub('[^A-Za-z0-9_]','',s))
    df = df.rename(columns={'Column Headings': 'Headings'
                            , 'Enumerated List': 'Enumerate'
                            , 'Data Entry Validations': 'Validations'
                            , 'Additional Help Text': 'Help'})
    lst.append(df[['SECTION','TABLE_NAME','COLUMN_NAME','Headings']])
df_csv = pd.concat(lst) 


engine = create_engine(connstr)
conn = engine.connect()
query = f""" \
    SELECT COLUMN_NAME, TABLE_NAME
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_NAME IN({tbls})
""" 
df_schema = pd.read_sql(query,con=conn)   
conn.close() 
df_schema.loc[df_schema['TABLE_NAME']=='EinsteinPublicIP']

,COLUMN_NAME,TABLE_NAME
18,PK_EinsteinPublicIP,EinsteinPublicIP
19,PK_OrgSubmission,EinsteinPublicIP
20,IPAddress,EinsteinPublicIP
21,Name,EinsteinPublicIP
22,ISP,EinsteinPublicIP
23,OtherISP,EinsteinPublicIP
24,ExternalOrg,EinsteinPublicIP
25,Visible,EinsteinPublicIP
26,UserId,EinsteinPublicIP
27,isActive,EinsteinPublicIP


In [108]:
df_csv.loc[df_csv['TABLE_NAME']=='EinsteinPublicIP']

,SECTION,TABLE_NAME,COLUMN_NAME,Headings
0,1,EinsteinPublicIP,StartingIP,Starting IP
1,1,EinsteinPublicIP,EndingIP,Ending IP
2,1,EinsteinPublicIP,TICMTIPS,TIC/MTIPS
3,1,EinsteinPublicIP,NamePurpose,Name/Purpose
4,1,EinsteinPublicIP,InternetServiceProviderISP,Internet Service Provider (ISP)
5,1,EinsteinPublicIP,OtherISP,Other ISP
6,1,EinsteinPublicIP,VisibletoEINSTEIN,Visible to EINSTEIN?
7,1,EinsteinPublicIP,ExternalOrganization,External Organization


In [99]:
df_merge = df_csv.merge(df_schema, left_on=['TABLE_NAME', 'COLUMN_NAME'], right_on=['TABLE_NAME', 'COLUMN_NAME'], suffixes=('_', ''))
df_merge

,Headings,Validations,Enumerate,Help,SECTION,TABLE_NAME,COLUMN_NAME
0,Other ISP,Optional Entry\n\nValidation\n- If the ?ISP? f...,NaN,If you selected ?Other? for ISP then enter the...,1,EinsteinPublicIP,OtherISP
1,External Organization,Optional Entry\n\nValidation\nSubstitute with ...,NaN,If this address range is associated with an ex...,2,EinsteinUnannounced,ExternalOrganization
2,BGP Peer IP,Mandatory Entry\n\nIPv4_Address Validation\nxx...,NaN,"BGP Peer IP information, OR the BGP Peer Name ...",3,EinsteinBGP,BGPPeerIP
3,Connection Type,Mandatory\nValidation\nMust choose one of the ...,Telco Termination\nModem Bank\nPartner Connect...,Select the type of unmonitored internet connec...,4,EinsteinUnmonitoredTraffic,ConnectionType
4,Other Connection Type,Optional Entry\n\nValidations\n- If the Connec...,NaN,"If you selected ?Other? for Connection Type, t...",4,EinsteinUnmonitoredTraffic,OtherConnectionType
5,Name/Purpose,Mandatory Entry\n\nValidation\nField is requir...,NaN,A short description of the unmonitored connect...,4,EinsteinUnmonitoredTraffic,NamePurpose
6,Other ISP,Optional Entry\n\nValidation\n- If the ?ISP? f...,NaN,If you selected ?Other? for ISP then enter the...,4,EinsteinUnmonitoredTraffic,OtherISP


In [ ]:
df-merge = df_schema.merge

In [ ]:
alt = ''
for i,r in df_csv.iterrows():
    TABLE_NAME = r['TABLE_NAME']
    for tname in [TABLE_NAME, f'{TABLE_NAME}SNAPSHOT']:
        alt = alt + f"""
        --ALTER TABLE {tname} DROP COLUMN {r['COLUMN_NAME']};
        IF NOT EXISTS(SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME='{tname}' and COLUMN_NAME='{r['COLUMN_NAME']}')
        BEGIN 
            ALTER TABLE {tname} ADD {r['COLUMN_NAME']} NVARCHAR(255) NULL
        END
        GO
        """
print( f'{alt}' )

In [106]:
alt = ''
for i,r in df_csv.iterrows():
    TABLE_NAME = r['TABLE_NAME']
    for tname in [TABLE_NAME, f'{TABLE_NAME}SNAPSHOT']:
        alt = alt + f"""
        --ALTER TABLE {tname} DROP COLUMN {r['COLUMN_NAME']};
        IF NOT EXISTS(SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME='{tname}' and COLUMN_NAME='{r['COLUMN_NAME']}')
        BEGIN 
            ALTER TABLE {tname} ADD {r['COLUMN_NAME']} NVARCHAR(255) NULL
        END
        GO
        """
print( f'{alt}' )


        --ALTER TABLE EinsteinPublicIP DROP COLUMN StartingIP;
        IF NOT EXISTS(SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME='EinsteinPublicIP' and COLUMN_NAME='StartingIP')
        BEGIN 
            ALTER TABLE EinsteinPublicIP ADD StartingIP NVARCHAR(255) NULL
        END
        GO
        
        --ALTER TABLE EinsteinPublicIPSNAPSHOT DROP COLUMN StartingIP;
        IF NOT EXISTS(SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME='EinsteinPublicIPSNAPSHOT' and COLUMN_NAME='StartingIP')
        BEGIN 
            ALTER TABLE EinsteinPublicIPSNAPSHOT ADD StartingIP NVARCHAR(255) NULL
        END
        GO
        
        --ALTER TABLE EinsteinPublicIP DROP COLUMN EndingIP;
        IF NOT EXISTS(SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME='EinsteinPublicIP' and COLUMN_NAME='EndingIP')
        BEGIN 
            ALTER TABLE EinsteinPublicIP ADD EndingIP NVARCHAR(255) NULL
        END
        GO
        
        --ALTER TABLE EinsteinPublicIPS

In [ ]:
tbls = '\'' + '\', \''.join( [s['name'] for s in secs] ) + '\''
print(tbls)


In [ ]:
engine = create_engine(connstr)
conn = engine.connect()
query = f""" \
    SELECT COLUMN_NAME, TABLE_NAME
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_NAME IN({tbls})
""" 
df_schema = pd.read_sql(query,con=conn)   
conn.close()
df_schema.head()

In [ ]:
df_new = df_csv.loc[ ~df_csv.COLUMN_NAME.isin(df_schema.COLUMN_NAME ) ]
df_new.head()

In [ ]:
df_rem = df_schema.loc[ ~df_schema.COLUMN_NAME.isin( df_csv.COLUMN_NAME ) ]
df_rem